<a href="https://colab.research.google.com/github/fabiobasson/my-repo/blob/main/oficios_qualidade.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Preciso criar um diretorio chamado laudo e jogar os oficios para dentro do mesmo (arquivos.pdf)
! mkdir -p laudo

from google.colab import files
uploaded = files.upload()

Saving Laudo_2628256_7_BUZ_33_RJS_22111210.pdf to Laudo_2628256_7_BUZ_33_RJS_22111210.pdf
Saving Laudo_2628563_7_RI_26_BA_221110214.pdf to Laudo_2628563_7_RI_26_BA_221110214.pdf


In [95]:
import warnings
warnings.filterwarnings('always')
warnings.filterwarnings('ignore')

# instalações e bibliotecas

#!git clone https://www.github.com/camelot-dev/camelot
#!cd camelot
!pip install "camelot-py[cv]"

!apt-get install ghostscript

!pip install 'PyPDF2<3.0'
!pip install xlwt

!pip install pdfminer.six
from pdfminer.high_level import extract_text

import pandas as pd
import io

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Reading package lists... Done
Building dependency tree       
Reading state information... Done
ghostscript is already the newest version (9.50~dfsg-5ubuntu4.7).
0 upgraded, 0 newly installed, 0 to remove and 34 not upgraded.
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [91]:
# Lendo documento . pdf e extraindo as tabela com a biblioteca Camelot

import camelot
tables = camelot.read_pdf('Laudo_2628256.pdf',header=None,index=False)
tables

# Criando Dataframe após extração das tabelas do arquivo .pdf

tabelas = tables[0].df[[1,0,2]].reset_index(drop=True)

In [93]:
tabelas.info();tabelas;tabelas.shape

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 14 entries, 0 to 13
Data columns (total 3 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   1       14 non-null     object
 1   0       14 non-null     object
 2   2       14 non-null     object
dtypes: object(3)
memory usage: 464.0+ bytes


(14, 3)

In [94]:
# Iniciando o ETL

# remover linhas e colunas desnecessárias a extração das informações

tabelas = tabelas.drop(0, axis='index')
tabelas = tabelas.drop(1, axis='index')

# Renomear colunas para o padrão do documento

tabelas.rename(columns = {1:'Norma Referencia',0:'Tipo Dados',2:'Situacao BDIEP'}, inplace = True)

tabelas

,Norma Referencia,Tipo Dados,Situacao BDIEP
2,RANP n° 880/2022,3.1. Arquivos Auxiliares - Esquemas de Ferrame...,NÃO CONFORME
3,RANP n° 880/2022,3.2. Arquivos Auxiliares - Dados Direcionais,APROVADO
4,RANP n° 880/2022,3.3. Arquivos Auxiliares - Relatório de Campo ...,
5,RANP n° 880/2022,3.4. Perfis Durante a Perfuração,NÃO CONFORME
6,RANP n° 880/2022,3.5. Perfis Convencionais,APROVADO
7,RANP n° 880/2022,3.6. Perfis Especiais - Teste de Formação a Cabo,APROVADO
8,RANP n° 880/2022,3.7. Perfis Especiais - Sísmica de Poço,
9,RANP n° 880/2022,3.8. Perfis Especiais - Amostrador Lateral,APROVADO
10,RANP n° 880/2022,3.9. Perfil Digital Processado,NÃO CONFORME
11,RANP n° 880/2022,3.10. Perfil Composto,


In [97]:
# Extraindo texto do documento com a biblioteca Extract_text

texto  = extract_text('Laudo_2628256.pdf', 'rb')

# Criando Dataframe após extração das tabelas do arquivo .pdf

texto = pd.read_fwf(io.StringIO(texto))

texto = texto[16:22]
texto

,"Agencia Nacional de Petróleo, Gás Natura e Biocombustíveis"
16,2211.1.0210
17,18/11/2022
18,7-BUZ-33-RJS
19,74316029460
20,13/03/2020
21,Petrobras


In [98]:
# Iniciando o ETL

# remover linhas e colunas desnecessárias a extração das informações


# Renomear colunas para o padrão do documento

texto.rename(columns = {'Agencia Nacional de Petróleo, Gás Natura e Biocombustíveis':' CABEÇALHO'}, inplace = True)

texto = texto.T
texto.rename(columns = {16:'Nº do Laudo de Reprovação', 17:'Data da Avaliação',18:'Nome do Poço',19:'Nº Cadastro do Poço',20:'Conclusão do Poço',21:'Empresa Operadora'}, inplace = True)


In [99]:
texto

,Nº do Laudo de Reprovação,Data da Avaliação,Nome do Poço,Nº Cadastro do Poço,Conclusão do Poço,Empresa Operadora
CABEÇALHO,2211.1.0210,18/11/2022,7-BUZ-33-RJS,74316029460,13/03/2020,Petrobras


In [102]:
# Concatenando o texto extraido e as tabelas extraidas

concatenar = pd.concat([texto,tabelas],ignore_index=True)
#concatenar = pd.concat([df3,tab])
concatenar.shape
concatenar

# Saida para um planilha excel

concatenar.to_excel('file_name.xls')

In [17]:
descproblema = ['Valor associado a menmonicos'],['Dados nao remetidos ao BDEP'],['Ausência de informação de perflagem conforme NPR '],['Valor inválido associado a mnemônicos '],['Cabeçalho '],['Valor inválido associado a mnemônicos'],['Ausência de informação de perflagem conforme NPR']
df4 = pd.DataFrame(descproblema,columns=['Descrição do Problema'])

df4 = df4.reset_index(drop=True)
df4.info()
#df4.rename(columns = {'16':'team_name', '17':'points_scored'}, inplace = True)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7 entries, 0 to 6
Data columns (total 1 columns):
 #   Column                 Non-Null Count  Dtype 
---  ------                 --------------  ----- 
 0   Descrição do Problema  7 non-null      object
dtypes: object(1)
memory usage: 184.0+ bytes


In [106]:
texto.loc[107:210].to_excel("texto1.xls")

TypeError: ignored

In [ ]:
search_for = "NÃO CONFORMIDADES" # What to search in the log lines...
line_num = 0 # Line number
lines_found = 0 # Quantity of lines found...
out_file = "saida.txt"
in_file = "texto.txt"

with open(out_file, 'w') as out_file:
    with open(in_file, "r") as in_file:
        for line in in_file:
            line_num += 1
            if search_for in line:
                lines_found += 1
                print("Found '{}' in line {}...".format(search_for, line_num))
                out_file.write(line)

        print("Found {} lines...".format(lines_found))

Found 'NÃO CONFORMIDADES' in line 107...
Found 'NÃO CONFORMIDADES' in line 114...
Found 'NÃO CONFORMIDADES' in line 122...
Found 'NÃO CONFORMIDADES' in line 149...
Found 'NÃO CONFORMIDADES' in line 186...
Found 5 lines...


In [ ]:
file = open('texto.txt')
content = file.readlines() 
content[105:114]


['104,Page 1 of 4\n',
 '105,"\x0c5. NÃO CONFORMIDADES REFERENTES À REMESSA DE DADOS, À EST"\n',
 '106,NOMENCLATURA DOS ARQUIVOS\n',
 '107,Itens:\n',
 '108,5.1. Remessa de Dados\n',
 '109,5.2. Estrutura de Diretórios\n',
 '110,5.3. Formato / Nomenclatura\n',
 '111,Não Conformidades:\n',
 '112,6. NÃO CONFORMIDADES REFERENTES AOS RELATÓRIOS E ARQUIVOS\n']

In [8]:
text.split('\n')[186:210]

AttributeError: ignored

In [3]:
!pip install tabula-py
!pip install tabulate

from tabula import read_pdf
from tabulate import tabulate
import pandas as pd
import io

# Read the only the page n°6 of the file
text = read_pdf('Laudo_2628256.pdf',pages = 0, 
                         multiple_tables = True, stream = True, encoding="utf-8")

# Transform the result into a string table format
table = tabulate(text)

# Transform the table into dataframe
df1 = pd.read_fwf(io.StringIO(table))
df1.info()
df1
# Save the final result as excel file
#df1[3:11].to_excel("oficios_qualidade1.xls")

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 30 entries, 0 to 29
Data columns (total 9 columns):
 #   Column                                                      Non-Null Count  Dtype 
---  ------                                                      --------------  ----- 
 0   ---------------------------------                           5 non-null      object
 1   ----------------------------------                          30 non-null     object
 2   -----------------------------------                         8 non-null      object
 3   -------------------------------                             5 non-null      object
 4   -------------------------------.1                           5 non-null      object
 5   -------------------------------.2                           5 non-null      object
 6   ----------------------------------------------------------  4 non-null      object
 7   -------------------------------------------------------     20 non-null     object
 8   -------------

,---------------------------------,----------------------------------,-----------------------------------,-------------------------------,-------------------------------.1,-------------------------------.2,----------------------------------------------------------,-------------------------------------------------------,-------------------------------------------------------.1
0,0 1.1. No do Laudo,0 1.2. Data da,0 1.3. Nome do,0 1.4. No Cadastro do,0 1.5. Conclusão do,0 1.6. Empresa,NaN,NaN,NaN
1,1 de Reprovação:,1 Avaliação:,1 Poço:,1 Poço:,1 Poço:,1 Operadora:,NaN,NaN,NaN
2,2 2211.1.0210,2 18/11/2022,2 7-BUZ-33-RJS,2 74316029460,2 13/03/2020,2 Petrobras,NaN,NaN,NaN
3,"Name: 1. CABEÇALHO, dtype: object","Name: Unnamed: 0, dtype: object","Name: Unnamed: 1, dtype: object","Name: Unnamed: 2, dtype: object","Name: Unnamed: 3, dtype: object","Name: Unnamed: 4, dtype: object",NaN,NaN,NaN
4,NaN,0 2.3. Código de Cadastro da(s),NaN,NaN,NaN,NaN,0 2.1. Número de Protocolo da(s) 2.2. Data ...,NaN,NaN
5,NaN,1 Mídia(s) no BDEP:,NaN,NaN,NaN,NaN,1 Remessa(s): Rem...,NaN,NaN
6,NaN,"Name: Unnamed: 0, dtype: object",NaN,NaN,NaN,NaN,"Name: 2. CONTROLE DE REMESSAS (USO INTERNO), d...",NaN,NaN
7,NaN,0 Situação dos Dados no BDEP*:,NaN,NaN,NaN,NaN,NaN,0 Tipos de Dados: Norma de Re...,NaN
8,NaN,1 NÃO CONFORME,NaN,NaN,NaN,NaN,NaN,1 3.1. Arquivos Auxiliares - Esquemas de F...,NaN
9,NaN,2 APROVADO,NaN,NaN,NaN,NaN,NaN,2 3.2. Arquivos Auxiliares - Dados Direcio...,NaN


In [7]:
import PyPDF2

pdf_file = open('Laudo_3047648.pdf','rb')

dados_pdf = PyPDF2.PdfFileReader(pdf_file)

pagina1 = dados_pdf.getPage(0)
texto_pagina1 = pagina1.extractText()

df2 = pd.read_fwf(io.StringIO(texto_pagina1))

print(texto_pagina1[180:228])

print(texto_pagina1[229:378])

ModuleNotFoundError: ignored

In [26]:
print(tables[0].parsing_report)
>>> {'accuracy': 100.0, 'whitespace': 0.0, 'order': 1, 'page': 1}
for i in range(tables.n):
    print(tables[i].parsing_report)
>>> {'accuracy': 100.0, 'whitespace': 0.0, 'order': 1, 'page': 1}
>>> {'accuracy': 100.0, 'whitespace': 0.0, 'order': 2, 'page': 1}

print(tables[0])

for i in range(tables.n):
    print(tables[i])

{'accuracy': 100.0, 'whitespace': 37.5, 'order': 1, 'page': 1}
{'accuracy': 100.0, 'whitespace': 37.5, 'order': 1, 'page': 1}
<Table shape=(14, 4)>
<Table shape=(14, 4)>


In [ ]:
print(dados_pdf.getPage(0))

{'/Type': '/Page', '/Parent': IndirectObject(2, 0, 139966990663536), '/Resources': IndirectObject(6, 0, 139966990663536), '/Contents': [IndirectObject(5, 0, 139966990663536)], '/MediaBox': [0, 0, 594, 842], '/CropBox': [0, 0, 594, 842]}


In [ ]:
page = pdf.pages[0]
page

<Page:1>

In [ ]:
tokenized_text = list(text)
print(tokenized_text)

['A', 'g', 'e', 'n', 'c', 'i', 'a', ' ', 'N', 'a', 'c', 'i', 'o', 'n', 'a', 'l', ' ', 'd', 'e', ' ', 'P', 'e', 't', 'r', 'ó', 'l', 'e', 'o', ',', ' ', 'G', 'á', 's', ' ', 'N', 'a', 't', 'u', 'r', 'a', ' ', 'e', ' ', 'B', 'i', 'o', 'c', 'o', 'm', 'b', 'u', 's', 't', 'í', 'v', 'e', 'i', 's', '\n', 'S', 'u', 'p', 'e', 'r', 'i', 'n', 't', 'e', 'n', 'd', 'ê', 'n', 'c', 'i', 'a', ' ', 'd', 'e', ' ', 'D', 'a', 'd', 'o', 's', ' ', 'T', 'é', 'c', 'n', 'i', 'c', 'o', 's', '\n', 'B', 'a', 'n', 'c', 'o', ' ', 'd', 'e', ' ', 'D', 'a', 'd', 'o', 's', ' ', 'd', 'e', ' ', 'E', 'x', 'p', 'l', 'o', 'r', 'a', 'ç', 'ã', 'o', ' ', 'e', ' ', 'P', 'r', 'o', 'd', 'u', 'ç', 'ã', 'o', '\n', 'L', 'a', 'u', 'd', 'o', ' ', 'd', 'e', ' ', 'A', 'v', 'a', 'l', 'i', 'a', 'ç', 'ã', 'o', ' ', 'd', 'e', ' ', 'D', 'a', 'd', 'o', 's', ' ', 'd', 'e', ' ', 'P', 'o', 'ç', 'o', '\n', '\n', '1', '.', ' ', 'C', 'A', 'B', 'E', 'Ç', 'A', 'L', 'H', 'O', '\n', '\n', '1', '.', '1', '.', ' ', 'N', 'º', ' ', 'd', 'o', ' ', 'L', 'a', 'u